In [1]:
import hilbertcurve
import nglview
import numpy as np
import py3Dmol
from hilbertcurve.hilbertcurve import HilbertCurve

import pytraj as pt 
import matplotlib.pyplot as plt
from matplotlib import cm


In [2]:
class DummyPDB:
    temp = "ATOM      1  Du  TMP     1       0.000   0.000   0.000  1.00  0.00";   # public 
    def __init__(self, vecX, vecY, vecZ, elems="", outname="tmp.pdb"):
        self.x=vecX
        self.y=vecY
        self.z=vecZ
        self.elems = elems

        self.outname = outname
#         print(self.outname)
        self.PDBlines()
        
        if len(self.x) != len(self.y) or len(self.x) != len(self.z):
            print("Length of dimention is not equil")
            
    def PDBlines(self):
        theline=""
        for i in range(len(self.x)):
            tmpcopy=self.temp;
            indlen = len(str(i));
            atmInd = "{:>5}".format(i+1)
            X = "{:>8}".format(self.x[i])
            Y = "{:>8}".format(self.y[i])
            Z = "{:>8}".format(self.z[i])
            try:
                Elem = self.elems[i];
            except:
                Elem = "Du";
            Elem = "{:>4}".format(Elem);
            tmpcopy = tmpcopy[:12]+Elem+tmpcopy[16:]
            tmpcopy = tmpcopy[:6]+atmInd+tmpcopy[11:]
            tmpcopy = tmpcopy[:30]+X+tmpcopy[38:]
            tmpcopy = tmpcopy[:38]+Y+tmpcopy[46:]
            tmpcopy = tmpcopy[:46]+Z+tmpcopy[54:]
            theline += tmpcopy+"\n"
        self.lines = theline;
        with open(self.outname,'w') as outfile1:
            outfile1.write(self.lines)
        return theline
    def show(self):
        viewer = nglview.show_file(self.outname)
        return viewer


In [3]:
with open("rec.pdb","r") as file1:
    pdbfile = [i.strip('\n') for i in  file1.readlines()] #.split('\n')
with open("trajLigand.mol2","r") as file2:
    molfile = [i.strip('\n') for i in file2.readlines()]
    
ite=4; 
dim=3;
cutoff=14;
distances = list(range(2**(ite*dim)))

HC1 = HilbertCurve(ite, dim)
points1 = HC1.points_from_distances(distances)

HC2 = HilbertCurve(6,2)
points2 = HC2.points_from_distances(distances)


In [9]:
traj = pt.load("rec.pdb")
traj.top.set_reference(traj[0])

proXYZ = traj.xyz[0][traj.top.select(":1-183")]
ovcenter = np.mean(traj.xyz[0][traj.top.select(":LIG <: 5 & !:LIG")], axis=0)
ligcenter = np.mean(traj.xyz[0][traj.top.select(":LIG")], axis = 0)

atomSel = traj.top.select(":LIG <: 5 & !:LIG")
COMLIG = pt.center_of_mass(traj=traj, mask=atomSel)[0]
print(ovcenter)
print(ligcenter)
print(COMLIG)


[ 0.55463636 -0.99151636  6.49759273]
[ 0.3896     -0.31993333  6.30606667]
[ 0.60004533 -1.02904721  6.38182702]


In [5]:
tmp=traj.top.atoms
select1 = traj.top.select(":LIG <@ 12 & !:LIG, T3P & !@CL,K ")
tmpelems = np.array([i.name for i in traj.top.atoms])[select1]
tmpxyz = traj.xyz[0][select1]
tmpx = tmpxyz[:,0]
tmpy = tmpxyz[:,1]
tmpz = tmpxyz[:,2]

points1 = np.array(points1)
points2 = np.array(points2)

x2 = points2[:,0]
y2 = points2[:,1] 

pointT = (points1) + (COMLIG-8)


c1=0; c2=0; c3=0
print("points", len(pointT))
print("tmpxyz", len(tmpxyz))

dum2elems = []
for i in range(len(pointT)):
    diff = pointT[i] - tmpxyz;
    sums = np.sum(np.square(diff), axis = 1); 
    distmin = np.round(np.sqrt(np.min(sums)),4)
    if (distmin < 2.5): #0.866):
        c1 += 1
        dum2elems.append(tmpelems[sums == np.min(sums)][0])
    elif (distmin > 1.732):
        c2 += 1
        dum2elems.append("Du")
    else :
        c3 += 1
        dum2elems.append("Du")
print(c1, c2, c3)



HCx = np.round(pointT[:,0], 4)
HCy = np.round(pointT[:,1], 4)
HCz = np.round(pointT[:,2], 4)


dummy = DummyPDB(HCx,HCy,HCz, elems=dum2elems, outname="tmp1.pdb")
dummy2 = DummyPDB(tmpx, tmpy, tmpz, elems=tmpelems, outname="tmp2.pdb")


points 4096
tmpxyz 744
3717 379 0


In [57]:
from numpy import rot90
class HilberItem:
    dim=3

    def __init__(self, ite=4, recenter=np.array([0,0,0])):
        self.distances = np.arange(2 ** (ite * self.dim));
        self.length = 2 ** ite
        self.HCObj = HilbertCurve(ite, dim)
        self.points = np.array(self.HCObj.points_from_distances(distances))
        print(type(self.points))
        self.x = self.points[:,0]
        self.y = self.points[:,1]
        self.z = self.points[:,2]
        self.matrix = self.distances.reshape((self.length,self.length,self.length))

        
    def reduce(self):
        pass
    def shift(self, center):
        self.points = (self.points + np.array(center) - self.length)
        self.x = self.points[:,0]
        self.y = self.points[:,1]
        self.z = self.points[:,2]
    def T(self, direction):
        pass
    
hilbert1 = HilberItem(4)
hilbert1.shift(COMLIG)
hilbert2 = np.rot90(hilbert1.matrix, 1, (0,2))
# Axes (0,2) horizontally rotate. 
# From first axes to the second axes. 

print(hilbert1.x)
print(hilbert1.points)
print(hilbert1.matrix)
print(hilbert2)

<class 'numpy.ndarray'>
[-15.39995467 -15.39995467 -15.39995467 ...  -0.39995467  -0.39995467
  -0.39995467]
[[-15.39995467 -17.02904721  -9.61817298]
 [-15.39995467 -17.02904721  -8.61817298]
 [-15.39995467 -16.02904721  -8.61817298]
 ...
 [ -0.39995467 -16.02904721  -8.61817298]
 [ -0.39995467 -17.02904721  -8.61817298]
 [ -0.39995467 -17.02904721  -9.61817298]]
[[[   0    1    2 ...   13   14   15]
  [  16   17   18 ...   29   30   31]
  [  32   33   34 ...   45   46   47]
  ...
  [ 208  209  210 ...  221  222  223]
  [ 224  225  226 ...  237  238  239]
  [ 240  241  242 ...  253  254  255]]

 [[ 256  257  258 ...  269  270  271]
  [ 272  273  274 ...  285  286  287]
  [ 288  289  290 ...  301  302  303]
  ...
  [ 464  465  466 ...  477  478  479]
  [ 480  481  482 ...  493  494  495]
  [ 496  497  498 ...  509  510  511]]

 [[ 512  513  514 ...  525  526  527]
  [ 528  529  530 ...  541  542  543]
  [ 544  545  546 ...  557  558  559]
  ...
  [ 720  721  722 ...  733  734  735]
  [

In [6]:
def assignElem(Hpoints, trajObj, refcutoff):
    c1=0; c2=0; c3=0
    maskStr = ":LIG <@ "+str(refcutoff)+" & !:LIG, T3P & !@CL,K "
    tmpselect = trajObj.top.select(maskStr)
    tmpelems = np.array([i.name for i in trajObj.top.atoms])[tmpselect]
    tmpxyz = trajObj.xyz[0][tmpselect]
    
    print("points", len(pointT))
    print("tmpxyz", len(tmpxyz))
    dum2elems = []
    for i in range(len(pointT)):
        
        diff = pointT[i] - tmpxyz;
        sums = np.sum(np.square(diff), axis = 1); 
        distmin = np.round(np.sqrt(np.min(sums)),4)
        if (distmin < 2.5): #0.866):
            c1 += 1
            dum2elems.append(tmpelems[sums == np.min(sums)][0])
        elif (distmin > 1.732):
            c2 += 1
            dum2elems.append("Du")
        else :
            c3 += 1
            dum2elems.append("Du")
    return dum2elems
    print(c1, c2, c3)
    
a=assignElem(pointT, traj, 12)
# print(a)

points 4096
tmpxyz 744


In [184]:
view = nglview.show_file("tmp1.pdb")
view.add_component("trajLigand.mol2")

# view.add_component("tmp2.pdb")

view.add_component("rec.pdb")
view



NGLWidget()

In [7]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [40]:

16*16*16

4096